In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier # Достаточно для прохождения лабы
import numpy as np
import xgboost as xgb # Достаточно для прохождения ачивки
#pip install --upgrade xgboost

In [3]:
def prepare_df(path_to_file: str, is_train: bool = True):
    # читаем исходный массив
    df = pd.read_csv(path_to_file)

    if is_train:
        # удалили строку с NaN'ами (в Y тоже был NaN)
        df = df.drop(df[(df['TARGET'].isnull())].index)

    # бестолковая колонка
    df = df.drop('CLNT_JOB_POSITION', axis=1)

    # удалили колонки с нулевой дисперсией
    df = df.drop(df.describe().T[(df.describe().T['std'] == 0)].index, axis=1)

    if is_train:
        # удалили 90% случайных значений с TARGET=0 чтобы уравновесить выборку
        df = df[df['TARGET'] == 0].sample(frac=.1).append(df[df['TARGET'] == 1])

        # выделяем из основного массива X и Y
        df_y = df['TARGET']
        df_x = df[df.columns.to_list()[:-1]]
    else:
        df_x = df

    non_text_columns = (df_x.dtypes != object)
    non_text_columns_names = non_text_columns[non_text_columns == True].index

    non_text_columns_names

    # заполнили нулями числовые колонки
    df_x[non_text_columns_names] = df_x[non_text_columns_names].fillna(0)

    # перекодировка для CLNT_TRUST_RELATION
    replace_map = {item: item for item in list(df_x['CLNT_TRUST_RELATION'].value_counts(dropna=False).index)}
    replace_mp_ = {'Друг': 'FRIEND', 
                   'Близкий ро': 'RELATIVE', 
                   'Дальний ро': 'RELATIVE',
                   'Мать': 'MOTHER',
                   'Брат': 'BROTHER',
                   'Отец': 'FATHER',
                   'Сестра': 'SISTER',
                   'Дочь': 'DAUGHTER',
                   'Сын': 'SON',
                   'Жена': 'RELATIVE',
                   'Муж': 'RELATIVE',
                   'мать': 'MOTHER',
                   'мама': 'MOTHER'
                  }
    for k, v in replace_mp_.items():
        replace_map[k] = v
    df_x['CLNT_TRUST_RELATION'] = df_x['CLNT_TRUST_RELATION'].map(replace_map)
    df_x['CLNT_TRUST_RELATION'].value_counts(dropna=False)

    df_x['APP_MARITAL_STATUS'] = df_x['APP_MARITAL_STATUS'].map(str.lower, na_action='ignore')
    df_x['APP_MARITAL_STATUS'].value_counts()

    # перекодировка для APP_MARITAL_STATUS
    replace_map = {item: item for item in list(df_x['APP_MARITAL_STATUS'].value_counts(dropna=False).index)}
    replace_mp_ = {' ': np.nan
                  }
    for k, v in replace_mp_.items():
        replace_map[k] = v
    df_x['APP_MARITAL_STATUS'] = df_x['APP_MARITAL_STATUS'].map(replace_map)
    df_x['APP_MARITAL_STATUS'].value_counts(dropna=False)

    df_x['APP_EDUCATION'] = df_x['APP_EDUCATION'].map(str.lower, na_action='ignore')
    df_x['APP_EDUCATION'].value_counts()


    # количество пропусков в колонке + тип колонки
    #with pd.option_context('display.max_rows', 150, 'display.max_columns', 150):  # more options can be specified also
    #    num_of_nulls = df_x.isnull().sum(axis=0)
    #    column_types = df_x.dtypes
    #    df_loc = pd.concat([num_of_nulls], axis=1)
    #    print(df_loc.join(pd.concat([column_types], axis=1), rsuffix='_other').sort_values(by=['0'], ascending=True))

    text_columns = (df_x.dtypes == object)
    text_columns_names = text_columns[text_columns == True].index.to_list()
    text_columns_names

    df_x = pd.get_dummies(data=df_x, columns=text_columns_names)
    
    if is_train:
        return df_x, df_y
    else:
        return df_x, None

In [4]:
#df_x_train, df_y_train = prepare_df('/mnt/Video/Лекции по машинному обучению/NewProLab/BigData 13/data/lab04data/lab04_train.csv', is_train=True)
df_x_train, df_y_train = prepare_df('/home/kit-kat/PycharmProjects/NPL/lab04/lab04data/lab04_train.csv', is_train=True)

/home/kit-kat/.virtualenvs/NovikovMS Lab1/lib/python3.6/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/kit-kat/.virtualenvs/NovikovMS Lab1/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kit-kat/.virtualenvs/NovikovMS Lab1/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [5]:
df_x_test, _ = prepare_df('/home/kit-kat/PycharmProjects/NPL/lab04/lab04data/lab04_test.csv', is_train=False)

In [28]:
tst_set = set(df_x_test.columns.to_list())
trn_set = set(df_x_train.columns.to_list())
print('tst - trn: {0}'.format(tst_set.difference(trn_set)))
print('trn - tst: {0}'.format(trn_set.difference(tst_set)))


tst - trn: set()
trn - tst: set()


In [21]:
# добавляем к выборке отсутствующие столбцы
df_x_test['PACK_108'] = 0  
df_x_test['APP_EDUCATION_ac'] = 0
df_x_test['APP_EDUCATION_av'] = 0


In [9]:
# рандом форест
dtc = RandomForestClassifier(n_estimators=100, max_depth=9, n_jobs=3, random_state=42)

dtc.fit(
    df_x_train,
    df_y_train
)
# value of your auc: 0.800140839615

RandomForestClassifier(max_depth=9, n_jobs=3, random_state=42)

In [36]:
# xg boost 
xgb_classifier = xgb.XGBClassifier(n_estimators = 100, max_depth=9, n_jobs=7, random_state=42)

xgb_classifier.fit(df_x_train, df_y_train)

# value of your auc: 0.837408909531

In [96]:
df_y_test = dtc.predict_proba(df_x_test)
df_y_train = dtc.predict(df_x_train)

In [38]:
df_y_test = xgb_classifier.predict_proba(df_x_test)
df_y_train = xgb_classifier.predict(df_x_train)

In [39]:
pd.DataFrame(df_y_test)

,0,1
0,0.069635,0.930365
1,0.999943,0.000057
2,0.999952,0.000048
3,0.999559,0.000441
4,0.999960,0.000040
...,...,...
44394,0.993066,0.006934
44395,0.067450,0.932550
44396,0.113779,0.886221
44397,0.688926,0.311074


In [40]:
result_df = pd.DataFrame(df_y_test)
result_df['ID'] = df_x_test['ID']

In [41]:
result_df

,0,1,ID
0,0.069635,0.930365,519130
1,0.999943,0.000057,234045
2,0.999952,0.000048,401256
3,0.999559,0.000441,551070
4,0.999960,0.000040,367285
...,...,...,...
44394,0.993066,0.006934,262122
44395,0.067450,0.932550,435918
44396,0.113779,0.886221,491359
44397,0.688926,0.311074,372969


In [42]:
result_df.columns = [0, 'target', 'id']

In [43]:
result_df[['id', 'target']].to_csv('/home/kit-kat/PycharmProjects/NPL/lab04/lab04data/lab04s.csv', index=False, sep='\t')


